## Target analysis of TA of WL-PSI of Scy6803

### Inspect data

In [ ]:
from cycler import cycler
from glotaran.io import load_parameters
from glotaran.io import save_result
from glotaran.optimization.optimize import optimize
from glotaran.project.scheme import Scheme
from pyglotaran_extras.inspect import show_a_matrixes
from pyglotaran_extras.plotting.plot_overview import plot_overview
from pyglotaran_extras.plotting.plot_traces import plot_fitted_traces
from pyglotaran_extras.plotting.plot_traces import select_plot_wavelengths

In [ ]:
from pyglotaran_extras import plot_data_overview

DATASETS = {
    "670TR1": "data/SCy6803WL/synWTred670_700nm_exc2RPnocycle1nm_reva.ascii",
    "670TR2": "data/SCy6803WL/synWTred670_700nm_exc2RPnocycle1nm_revb.ascii",
    "700TR1": "data/SCy6803WL/synWTred670_700nm_exc2RPnocycle1nm_revc.ascii",
    "700TR2": "data/SCy6803WL/synWTred670_700nm_exc2RPnocycle1nm_revd.ascii",
    "Red1SADS": "data/SCy6803WL/synWTred670_700nm_exc2RPnocycle1nm_reve.ascii",
    "WLRCSADS": "data/SCy6803WL/synWTred670_700nm_exc2RPnocycle1nm_revf.ascii",
    "Red2SADS": "data/SCy6803WL/synWTred670_700nm_exc2RPnocycle1nm_revg.ascii",
    "WLRP1SADS": "data/SCy6803WL/synWTred670_700nm_exc2RPnocycle1nm_revh.ascii",
}

fig, axes = plot_data_overview(
    DATASETS["670TR1"], nr_of_data_svd_vectors=3, linlog=True, linthresh=0.1
)
# change color map seismic or bwr
# axes[0].set_cmap('seismic')

## Target Analysis

### Used model and parameters

In [ ]:
target_model_path = "models/20230521model_PSI_TA_SCy6803WL.yml"

In [ ]:
target_parameters_path = "models/20230521optimized_parameters.csv"
parameters = load_parameters(target_parameters_path)  # load optimized parameters

#### Model file

In [ ]:
# Uncomment the following 2 lines to display the target model file in the notebook
# from glotaran.utils.ipython import display_file
# display_file(target_model_path, syntax="yaml")

# Alternatively (recommended), open the file in a text editor to see the model definition

#### Parameters file

In [ ]:
# Uncomment the next line and run the cell to print the starting values of the analysis

# parameters

### Create scheme and optimize it

In [ ]:
target_scheme = Scheme(
    model=target_model_path,  # type: ignore
    parameters=parameters,
    maximum_number_function_evaluations=15,
    clp_link_tolerance=0.1,
    data=DATASETS,  # type: ignore
)
target_scheme.validate()

In [ ]:
target_result = optimize(target_scheme, raise_exception=True)

To save the results of the optimization we can use the `save_result` command.

Because it saves *everything* it consumes about 50MB of disk space per save.

In [ ]:
save_result(
    result=target_result,
    result_path="results/20230520/result.yaml",
    allow_overwrite=True,
)

### Results and parameters

In [ ]:
# Just call the result to get the optimization result summary.
target_result
# For easier copy-and-paste try:
# print(target_result)

In [ ]:
# Access the result's `optimized_parameters` to print a markdown table of the optimized parameters:
target_result.optimized_parameters

### Amplitude matrices

In [ ]:
show_a_matrixes(target_result)

# Result plots

<sub>Note: The color scheme of the plots in this notebook may not match published figures.</sub>

## Fit quality

In [ ]:
target_result_TA = (
    target_result.data["670TR1"],
    target_result.data["670TR2"],
    target_result.data["700TR1"],
    target_result.data["700TR2"],
)
wavelengths = select_plot_wavelengths(target_result_TA, equidistant_wavelengths=True)
plot_fitted_traces(target_result_TA, wavelengths, linlog=True, linthresh=1);

The above command `plot_fitted_traces` is used to plot a selection of traces for a set of wavelengths (autogenerated using the `select_plot_wavelengths` function).
To show to make a manual selection of traces, and 'dress up the plot' see the code below, which reproduces Figure 2 of the paper.

In [ ]:
# Reproduction of Figure 2 of the paper
import warnings

from pyglotaran_extras.plotting.style import ColorCode as cc

with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    fig, ax_ = plot_fitted_traces(
        target_result_TA,
        [685, 700, 720, 760],
        linlog=True,
        linthresh=1,  # published figure uses 0.3 for easthetic reasons, but here 1 looks better
        axes_shape=(2, 2),
        figsize=(6, 4),
        title="",
        per_axis_legend=True,
        cycler=cycler(
            color=[
                cc.grey,
                cc.black,
                cc.grey,
                cc.black,
                cc.orange,
                cc.red,
                cc.orange,
                cc.red,
            ]
        ),
    )
    handles, labels = ax_.flatten()[0].get_legend_handles_labels()
    for i in range(len(handles)):
        if i == 1:
            labels[i] = "670 nm excitation"
        elif i == 5:
            labels[i] = "700 nm excitation"
        else:
            labels[i] = "_Hidden"
    for idx, ax in enumerate(ax_.flatten()):
        ax.set_ylabel(ax.title.get_text().replace("spectral = ", ""))
        if idx > 1:
            ax.set_xlabel("Time (ps)")
        else:
            ax.set_xlabel("")
        ax.set_title("")
        if ax.get_legend() is not None:
            ax.get_legend().remove()
        for line in ax.lines:
            line.set_linewidth(0.5)  # Set the line width here
    fig.legend(
        handles,
        labels,
        bbox_to_anchor=(0.5, -0.05),
        loc="lower center",
        ncol=len(handles),
    )
    fig.tight_layout()

## Overview 670 exc

In [ ]:
plot_overview(
    target_result.data["670TR1"],
    nr_of_data_svd_vectors=4,
    nr_of_residual_svd_vectors=1,
    linlog=False,
    linthresh=1,
    cycler=cycler(color=["y", "g", "tab:orange", "r", "k", "c", "b", "m", "tab:purple"]),
);

In [ ]:
plot_overview(
    target_result.data["670TR2"],
    nr_of_data_svd_vectors=4,
    nr_of_residual_svd_vectors=1,
    linlog=False,
    linthresh=1,
    cycler=cycler(color=["y", "g", "tab:orange", "r", "k", "c", "b", "m", "tab:purple"]),
);

## Overview 700 exc

In [ ]:
plot_overview(
    target_result.data["700TR1"],
    nr_of_data_svd_vectors=4,
    nr_of_residual_svd_vectors=1,
    linlog=False,
    linthresh=1,
    cycler=cycler(color=["y", "g", "tab:orange", "r", "k", "c", "b", "m", "tab:purple"]),
);

In [ ]:
plot_overview(
    target_result.data["700TR2"],
    nr_of_data_svd_vectors=4,
    nr_of_residual_svd_vectors=1,
    linlog=False,
    linthresh=1,
    cycler=cycler(color=["y", "g", "tab:orange", "r", "k", "c", "b", "m", "tab:purple"]),
);

## Comparison of the estimated SADS (orange) and the guidance spectra (blue)
The guidance spectra are (smooth) shapes derived elsewhere 

In [ ]:
target_result.data["Red1SADS"].data.plot()
target_result.data["Red1SADS"].fitted_data.plot()
target_result.data["Red2SADS"].data.plot()
target_result.data["Red2SADS"].fitted_data.plot();

In [ ]:
target_result.data["WLRP1SADS"].data.plot()
target_result.data["WLRP1SADS"].fitted_data.plot();

In [ ]:
target_result.data["WLRCSADS"].data.plot()
target_result.data["WLRCSADS"].fitted_data.plot();